# Creating a Cutout with the SARAH-2 dataset

This walkthrough describes the process of creating a cutout using the [SARAH-2 dataset by EUMETSAT](https://wui.cmsaf.eu/safira/action/viewDoiDetails?acronym=SARAH_V002_01).

The SARAH-2 dataset contains extensive information on solar radiation variables, like surface incoming direct radiation (SID) or surface incoming shortwave radiation (SIS).
It serves as an addition to the ERA5 dataset and as such requires the `cdsapi` to be setup properly.

> **Recommendation**
>
> This is a reduced version for cutout creation. Creating cutouts with ERA-5 is simpler and explained in more details.
> We therefore recommend you have a look at [this example first](https://atlite.readthedocs.io/en/latest/examples/create_cutout.html).

> **Note**:
>
> For creating a cutout from this dataset, you need to download large files and your computers memory needs to be able to handle these as well.

## Downloading the data set

To download the dataset, head to the EUMETSTATs website (the link points to the current 2.1 edition)

https://wui.cmsaf.eu/safira/action/viewDoiDetails?acronym=SARAH_V002_01 

On the bottom, select the products you want to include in the cutout, i.e. for us:

| variable | time span | time resolution | 
| --- | --- | --- |
| Surface incoming direct radiation (SID) | 2013 | Instantaneous |
| Surface incoming shortwave radiation (SIS) | 2013 | Instantaneous |

* Add each product to your cart and register with the website.
* Follow the instructions to activate your account, confirm your order and wait for the download to be ready.
* You will be notified by email with the download instructions.
* Download the ordered files of your order into a directory, e.g. `sarah-2`.
* Extract the `tar` files (e.g. for linux systems `tar -xvf *` or with `7zip` for windows) into the same folder

You are now ready to create cutouts using the SARAH-2 dataset.

## Specifying the cutout

Import the package and set recommended logging settings:

In [1]:
import atlite

import logging

logging.basicConfig(level=logging.INFO)

Disable urllib3 and cdsapi warnings.


In [2]:
cutout = atlite.Cutout(
    path="western-europe-2011-01.nc",
    module=["sarah", "era5"],
    sarah_dir="/home/vres-climate/data/sarah_v2",
    x=slice(-13.6913, 1.7712),
    y=slice(49.9096, 60.8479),
    time="2013-01",
    chunks={"time": 100},
)

INFO:atlite.cutout:Building new cutout western-europe-2011-01.nc


Let's see what the available features that is the available weather data variables are.

In [3]:
cutout.available_features.to_frame()

variables
module feature                      
sarah  influx          influx_direct
       influx         influx_diffuse
era5   height                 height
       wind                  wnd100m
       wind                roughness
       influx             influx_toa
       influx          influx_direct
       influx         influx_diffuse
       influx                 albedo
       temperature       temperature
       temperature  soil temperature
       runoff                 runoff

## Preparing the Cutout

No matter which dataset you use, this is where all the work actually happens.
This can be fast or take some or a lot of time and resources, among others depending on
your computer ressources (especially memory for SARAH-2).

In [4]:
cutout.prepare()

INFO:atlite.data:Calculating and writing with module sarah:


[########################################] | 100% Completed |  3min 19.2s


INFO:atlite.data:Calculating and writing with module era5:
INFO:atlite.datasets.era5:Downloading data for feature 'height' to /tmp/tmpk92au7s_.
INFO:atlite.datasets.era5:Downloading data for feature 'wind' to /tmp/tmpk92au7s_.


[########################################] | 100% Completed | 11.2s


INFO:atlite.datasets.era5:Downloading data for feature 'influx' to /tmp/tmpk92au7s_.


...

INFO:atlite.datasets.era5:Downloading data for feature 'temperature' to /tmp/tmpk92au7s_.


[########################################] | 100% Completed | 10.9s


INFO:atlite.datasets.era5:Downloading data for feature 'runoff' to /tmp/tmpk92au7s_.


[########################################] | 100% Completed | 10.8s
[########################################] | 100% Completed |  1.3s


<Cutout "western-europe-2011-01">
 x = -13.50 ⟷ 1.75, dx = 0.25
 y = 50.00 ⟷ 60.75, dy = 0.25
 time = 2013-01-01 ⟷ 2013-01-31, dt = H
 prepared_features = ['influx', 'height', 'wind', 'temperature', 'runoff']

Querying the cutout gives us basic information on which data is contained and can already be used.

## Inspecting the Cutout

In [5]:
cutout  # basic information

<Cutout "western-europe-2011-01">
 x = -13.50 ⟷ 1.75, dx = 0.25
 y = 50.00 ⟷ 60.75, dy = 0.25
 time = 2013-01-01 ⟷ 2013-01-31, dt = H
 prepared_features = ['influx', 'height', 'wind', 'temperature', 'runoff']

In [6]:
cutout.data.attrs  # cutout meta data

{'module': ['sarah', 'era5'],
 'prepared_features': ['influx', 'runoff', 'height', 'wind', 'temperature'],
 'chunksize_time': 100,
 'sarah_dir': '/home/vres-climate/data/sarah_v2'}

In [7]:
cutout.prepared_features  # included weather variables

module  feature    
sarah   influx            influx_direct
        influx           influx_diffuse
era5    height                   height
        wind                    wnd100m
        wind                  roughness
        influx               influx_toa
        influx                   albedo
        temperature         temperature
        temperature    soil temperature
        runoff                   runoff
Name: variables, dtype: object

In [8]:
cutout.data  # access to underlying xarray data

<xarray.Dataset>
Dimensions:           (time: 744, x: 62, y: 44)
Coordinates:
    lon               (x) float64 dask.array<chunksize=(62,), meta=np.ndarray>
    lat               (y) float64 dask.array<chunksize=(44,), meta=np.ndarray>
  * y                 (y) float64 50.0 50.25 50.5 50.75 ... 60.25 60.5 60.75
  * time              (time) datetime64[ns] 2013-01-01 ... 2013-01-31T23:00:00
  * x                 (x) float64 -13.5 -13.25 -13.0 -12.75 ... 1.25 1.5 1.75
Data variables:
    influx_direct     (time, y, x) float32 dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    influx_diffuse    (time, y, x) float32 dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    height            (y, x) float32 dask.array<chunksize=(44, 62), meta=np.ndarray>
    wnd100m           (time, y, x) float32 dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    roughness         (time, y, x) float32 dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    influx_toa        (time, y, x) float32 dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    albedo            (time, y, x) float32 dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    temperature       (time, y, x) float32 dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    soil temperature  (time, y, x) float32 dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    runoff            (time, y, x) float32 dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
Attributes:
    module:             ['sarah', 'era5']
    prepared_features:  ['influx', 'runoff', 'height', 'wind', 'temperature']
    chunksize_time:     100
    sarah_dir:          /home/vres-climate/data/sarah_v2